# Branch-and-Bound Algorithm za Maximum Leaf Spanning Tree Problem (MLSTP)

### Oznake koje cemo koristiti u algoritmu
- $G = (V, E)$
- $S_1, \overline{S_1}, S_0, \overline{S_0}, F, \overline{F} \subset V$
- $S_1$ - sme da sadrzi samo listove
- $S_0$ - ne sme da sadrzi listove
- $F = V \setminus (S_1 \cup S_0)$

### Koraci Algoritma

**1. Incijalizacija**
- Pokreni BFS algoritam iz svakog čvora
- Za svako dobijeno stablo izračunaj broj listova i \
  najmanju od tih vrednosti postavi kao inicijalnu donju granicu (LB)

**2. Odabir potproblema (radi dalje pretrage/istrage)**
- Održavaj listu (L) potproblema $(S_1, S_0, F)$ (inicijalno $S_1, S_0 = \emptyset, F = V$)
- Ako je lista prazna, završi algoritam
- Inače, odaberi potproblem $(S_1, S_0, F)$ sa vrha hipa
- Ako je $|S_1 \cup F| \lt LB$, odbaci potproblem

**3. Provera da li je potproblem dopustiv**
- Proveri da li u podgrafu polaznog grafa koji se sastoji samo od čvorova iz $F$ i $S_1$ postoji MST, tj. da li je povezan
- Ako ne postoji, tj. ako je potproblem nedopustiv, odbaci ga i nastavi dalje

**4. Ažuriranje donje granice (ako je moguće)**
- Za svaki čvor u $S_1 \cup F$ pokušaj da napraviš MST tako što pokreneš BFS iz tog čvora
- Ako to rezultira drvetom sa najvećim brojem listova do sada, ažuriraj LB

**5. Računanje gornje granice**
- Reši relaksiran potproblem tako što ćeš pronaci MST podgrafa polaznog grafa koji se sastoji samo od čvorova iz $F$ i $S_1$
- Ako je izračunata gornja granica $UB \le LB$, odbaci potproblem

**6. Grananje (Branching)**
- Izaberi čvor $v$ iz $F$ koji maksimizuje stepen u grafu
- Kreiraj 2 potproblema
  - Dodaj $v$ u skup $S_1$ (čime ga forsiraš da bude list) i kreiraj potproblem $(S_1, \overline{S_0}, \overline{F})$
  - Dodaj $v$ u skup $S_0$ (čime ga forsiraš da ne bude list)  i kreiraj potproblem $(\overline{S_1}, S_0, \overline{F})$
- Dodaj ove potprobleme u listu `L`

**7. Kraj**
- Zavrsi algoritam kad ne postoji vise potproblema u `L`
- Rešenje pronadjeno tokom ovog procesa je optimalno rešenje MLST problema

**Referenca**
- Fujie, T. (2003). "An Exact Algorithm for the Maximum Leaf Spanning Tree Problem." Computers & Operations Research, 30(2003), 1931-1944.


In [9]:
import heapq
import networkx as nx

def count_leaves(tree):
    return sum(1 for node in tree.nodes if tree.degree[node] == 1)

def compute_upper_bound(subgraph):
        mst = nx.minimum_spanning_tree(subgraph)
        return count_leaves(mst)

def branch_and_bound_mlstp(graph):

    # 1. Inicijalizacija
    best_tree = None
    best_leaf_count = 0
    for node in graph.nodes:
        bfs_tree = nx.bfs_tree(graph, source=node)
        leaf_count = count_leaves(bfs_tree)
        if leaf_count > best_leaf_count:
            best_tree = bfs_tree
            best_leaf_count = leaf_count

    lower_bound = best_leaf_count

    # kreiranje liste potproblema
    L = []
    root_problem = (set(), set(), set(graph.nodes))  # (S1, S0, F)
    heapq.heappush(L, (-lower_bound, root_problem))

    best_solution = None

    # 2. Odabir potproblema
    while L:
        _, (S1, S0, F) = heapq.heappop(L)

        # 3. Provera da li je potproblem dopustiv
        subgraph = graph.subgraph(F.union(S1))
        if not nx.is_connected(subgraph):
            continue

        # 4. Azuriranje donje granice
        for node in F:
            bfs_tree = nx.bfs_tree(graph.subgraph(F.union(S1)), source=node)
            leaf_count = count_leaves(bfs_tree)
            if leaf_count > best_leaf_count:
                best_leaf_count = leaf_count
                best_tree = bfs_tree

        # 5. Racunanje gornje granice
        upper_bound = compute_upper_bound(subgraph)
        if upper_bound <= best_leaf_count:
            continue

        # 6. Grananje (Branching)
        if F:
            v = max(F, key=lambda u: graph.degree[u])
            new_S1 = S1.union({v})
            new_S0 = S0.union({v})
            new_F = F.difference({v})

            heapq.heappush(L, (-upper_bound, (new_S1, S0, new_F)))
            heapq.heappush(L, (-upper_bound, (S1, new_S0, new_F)))

    return best_tree, best_leaf_count
